# 使用 Unsloth 对 DeepSeek-R1-Distill-Qwen-1.5B 模型进行 LoRA 微调

本 Notebook 展示了如何使用 `unsloth` 库对 `deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B` 模型进行高效的 QLoRA (Low-Rank Adaptation) 微调。

整个流程包括：
1.  环境准备与库导入
2.  加载预训练模型和分词器 (Tokenizer)。
3.  在微调前，对模型进行简单的推理测试。
4.  下载和格式化训练数据集
5.  使用 `unsloth` 的 `FastLanguageModel` 来为模型添加 LoRA 适配器。
6.  配置 `SFTTrainer` 监督微调训练配置。
7.  启动训练，并观察 Loss 变化情况
8.  保存微调后的模型
9.  测试训练后的生成结果

### 1. 环境准备与库导入

首先，我们需要安装并导入所有必要的库。`transformers` 用于加载模型和分词器，`unsloth` 用于高效微调，`trl` 提供了 `SFTTrainer`，而 `datasets` 用于处理数据。

**注意**: 在运行此 Notebook 之前，请确保已安装所有依赖包：
```bash
pip install -r requirements.txt
```

In [3]:
import torch
from unsloth import FastLanguageModel
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GenerationConfig, DataCollatorForSeq2Seq
from datasets import Dataset
import subprocess
import os

subprocess.run("export HF_ENDPOINT=https://hf-mirror.com", shell=True, capture_output=True, text=True)

CompletedProcess(args='export HF_ENDPOINT=https://hf-mirror.com', returncode=0, stdout='', stderr='')

### 2. 加载预训练模型和分词器 (Tokenizer)

In [5]:
# 定义模型和一些基本参数
max_seq_length = 8192
dtype = None # None 表示自动选择 (Float16 a T4, V100, BFloat16 a Ampere)
load_in_4bit = True # 使用 4bit 量化加载

# 这是您的模型标识符，请替换为您正在使用的模型
# 例如："qwen-1.5b_lora_model"
# model_name = "qwen-1.5b_lora_model" 
model_name = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B" 
# model_name = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit" 

# 这一步会返回一个经过 Unsloth 优化的模型和一个分词器
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.8.5: Fast Qwen2 patching. Transformers: 4.55.2.
   \\   /|    NVIDIA GeForce RTX 4090 D. Num GPUs = 1. Max memory: 23.643 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.81G [00:00<?, ?B/s]

  2025-09-02T14:13:26.826106Z  WARN  Reqwest(reqwest::Error { kind: Request, url: "https://transfer.xethub.hf.co/xorbs/default/1902d4009db2f761788e06de04673ca2f3cbf6961213739904b6f47e04186378?X-Xet-Signed-Range=bytes%3D24394871-24402531&Expires=1756826004&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly90cmFuc2Zlci54ZXRodWIuaGYuY28veG9yYnMvZGVmYXVsdC8xOTAyZDQwMDlkYjJmNzYxNzg4ZTA2ZGUwNDY3M2NhMmYzY2JmNjk2MTIxMzczOTkwNGI2ZjQ3ZTA0MTg2Mzc4P1gtWGV0LVNpZ25lZC1SYW5nZT1ieXRlcyUzRDI0Mzk0ODcxLTI0NDAyNTMxIiwiQ29uZGl0aW9uIjp7IkRhdGVMZXNzVGhhbiI6eyJBV1M6RXBvY2hUaW1lIjoxNzU2ODI2MDA0fX19XX0_&Signature=YPqhDzxpk8iYuNqQCTjvMHfJWCoeYdOaTfMQ7B3wUX7~9uSSBCFO1V08R3gksmx4ln-S5Jn7fnetzmcmilyF-y76NJeaZnB1GnAi-PKVl~WGejyi7DSweZ8Euvy2RPUfcLP1zil~dg-otsVdaVYeHElWNAypoXX~m0SfntPiDZ~RBWtOXs0DLl0YbW~oFJps1sR7SMn1Jt5RJwPKHpslT56uDj-uuWFx8xpgeLBzet0Cfq2JYD41HzMJiVEEtEqn~PBP~p-GbsbfTmtK07tw6XLexkflYqRibx9byDI6Ulv60rVISpGgTZEQrpgQLL3l8siW5z-Xdlce~FKLCJwAyg__&Key-Pair-Id=K2L8F4GPSG1IFC", source: hyper_util::client::

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

### 3. 微调前推理测试

在对模型进行任何修改之前，我们先用它来生成一段文本，看看原始模型的表现如何。这可以作为我们微调效果的基准参考。

In [6]:
# 模型推理的 Prompt 模板
inference_prompt = """以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
{}

### Response:
<think>{}
"""

In [7]:
FastLanguageModel.for_inference(model)

question = "男，28岁，程序员，最近一周每天工作到半夜，感觉头晕、脖子疼，有时候还恶心。"

inputs = tokenizer([inference_prompt.format(question, "")], return_tensors="pt").to("cuda")
attention_mask = inputs.input_ids.ne(tokenizer.pad_token_id).long().to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)

In [8]:
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [9]:
print(response[0].split("### Response:")[1])


<think>
好，我来仔细分析一下这个问题。首先，用户是一名28岁的男性程序员，最近一周每天工作到半夜，表现出头晕、脖子疼和 sometimes恶心等症状。需要根据这些症状和背景，进行临床推理、诊断和治疗。

首先，头晕和脖子疼可能与长时间的工作有关，尤其是程序员长时间使用电脑可能引发的脑力负担。脖子疼可能是因为长时间的使用或紧张，导致肌肉紧张，进而出现疼痛。

恶心可能是由于长期的使用电子设备引起的胃部不适，或者可能与压力或焦虑有关。不过，恶心通常是较轻微的，通常不会持续太久，所以需要进一步考虑。

接下来，考虑患者的年龄是28岁，属于中年，通常健康状况良好，但需要综合考虑。程序员的工作环境可能包括电脑、电子设备等，这些设备可能对患者的神经和肌肉造成一定的压力，甚至可能对神经系统的功能产生影响。

另外，长时间的工作可能导致心理压力，影响神经系统的正常功能，从而引发以上症状。此外，可能需要考虑是否存在其他潜在的疾病，比如脑膜炎、脑水肿等，这些在长时间使用电子设备后可能更容易发展。

在诊断方面，可能需要考虑以下几点：

1. **头晕和脖子疼**：可能与脑力紧张有关，建议进行脑部影像学检查，比如CT或MRI，以评估是否存在脑损伤或脑部功能异常。

2. **恶心**：可能需要进一步评估是否有胃部功能异常，或者是否存在胃炎或其他胃肠道疾病。

3. **神经功能评估**：如果以上检查不明确，可能需要进行脑电图或脑机接口研究，以评估神经系统的功能是否正常。

4. **心理因素**：考虑到程序员的工作环境和长期压力，可能需要进行相关心理评估，评估患者是否存在心理问题，如焦虑、抑郁等，这些可能影响症状的出现。

在治疗方面，可能需要综合考虑以下措施：

1. **神经功能评估**：通过CT、MRI等影像学检查，评估是否存在脑损伤或功能异常，必要时进行脑电图或脑机接口研究。

2. **心理治疗**：针对心理因素，如果患者表现出焦虑或抑郁，可能需要进行心理辅导或治疗，以缓解其症状。

3. **预防措施**：如果发现可能存在脑损伤或神经功能异常，建议采取预防措施，如延长工作时间，减少电子设备的使用，或者改变工作环境，以减轻长期压力和功能损伤的影响。

4. **药物治疗**：如果神经功能评估不明确，可能需要考虑药物治疗，如抗恶心药物、抗焦虑药物等，帮助缓解症状。

需

---

### 4. 下载和格式化训练数据集


医学推理数据集：https://huggingface.co/datasets/FreedomIntelligence/medical-o1-reasoning-SFT/viewer/zh

![dataset](images/dataset.png)

In [10]:
# 模型训练的 Prompt 模板
train_prompt = """以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
{}

### Response:
<think>
{}
</think>
{}
"""

In [11]:
EOS_TOKEN = tokenizer.eos_token # 添加 EOS Token

def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        # 将 EOS Token 添加到样本最后
        text = train_prompt.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "zh", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

README.md:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

medical_o1_sft_Chinese.json:   0%|          | 0.00/50.6M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/20171 [00:00<?, ? examples/s]

In [12]:
dataset[0]["text"]

'以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。\n请撰写一份恰当的回复，以完成该请求。\n在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。\n\n\n### Instruction:\n你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。\n请回答以下医学问题。\n\n### Question:\n根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？\n\n### Response:\n<think>\n这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。\n\n用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。\n\n但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。\n\n等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。\n\n仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。\n\n回到基本的症状描述上看，这种长期不愈合又复杂的状况，如果结合中医更偏重的病名，是不是有可能是涉及更深层次的感染？\n\n再考虑一下，这应该不是单纯的瘰疬，得仔细分析头皮增厚并出现空洞这样的严重症状。中医里头，这样的表现可能更符合‘蚀疮’或‘头疽’。这些病名通常描述头部严重感染后的溃烂和组织坏死。\n\n看看季节和孩子的体质，夏天又湿又热，外邪很容易侵入头部，对孩子这么弱的免疫系统简直就是挑战。头疽这个病名听起来真是切合，因为它描述的感染严重，溃烂到出现空洞。\n\n不过，仔细琢磨后发现，还有个病名似乎更为合适，叫做‘蝼蛄疖’，这病在中医里专指像这种严重感染并伴有深部空洞的情况。它也涵盖了化脓和皮肤增厚这些症状。\n\n

In [13]:
from IPython.display import display, Markdown

display(Markdown(dataset[0]["text"]))

以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？

### Response:
<think>
这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。

用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。

但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。

等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。

仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。

回到基本的症状描述上看，这种长期不愈合又复杂的状况，如果结合中医更偏重的病名，是不是有可能是涉及更深层次的感染？

再考虑一下，这应该不是单纯的瘰疬，得仔细分析头皮增厚并出现空洞这样的严重症状。中医里头，这样的表现可能更符合‘蚀疮’或‘头疽’。这些病名通常描述头部严重感染后的溃烂和组织坏死。

看看季节和孩子的体质，夏天又湿又热，外邪很容易侵入头部，对孩子这么弱的免疫系统简直就是挑战。头疽这个病名听起来真是切合，因为它描述的感染严重，溃烂到出现空洞。

不过，仔细琢磨后发现，还有个病名似乎更为合适，叫做‘蝼蛄疖’，这病在中医里专指像这种严重感染并伴有深部空洞的情况。它也涵盖了化脓和皮肤增厚这些症状。

哦，该不会是夏季湿热，导致湿毒入侵，孩子的体质不能御，其病情发展成这样的感染？综合分析后我觉得‘蝼蛄疖’这个病名真是相当符合。
</think>
从中医的角度来看，你所描述的症状符合“蝼蛄疖”的病症。这种病症通常发生在头皮，表现为多处结节，溃破流脓，形成空洞，患处皮肤增厚且长期不愈合。湿热较重的夏季更容易导致这种病症的发展，特别是在免疫力较弱的儿童身上。建议结合中医的清热解毒、祛湿消肿的治疗方法进行处理，并配合专业的医疗建议进行详细诊断和治疗。
<｜end▁of▁sentence｜>

### 5. 使用 Unsloth 添加 LoRA 适配器

这是使用 `unsloth` 的核心步骤。我们调用 `FastLanguageModel.get_peft_model`，它会非常高效地为模型注入 LoRA 模块。

- `r`: LoRA 的秩 (rank)，是控制模型复杂度和参数量的关键超参数。
- `target_modules`: 指定要在哪些线性层（如注意力机制的 q, k, v, o 投影层）上应用 LoRA。
- `lora_alpha`: LoRA 的缩放因子，通常设置为 `r` 的两倍或与 `r` 相同。
- `use_gradient_checkpointing`: 一种节省显存的技术，对于训练大模型至关重要。

In [14]:
# 因为 `model` 对象现在是由 Unsloth 创建的，它包含了所有必需的属性
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
      "q_proj",
      "k_proj",
      "v_proj",
      "o_proj",
      "gate_proj",
      "up_proj",
      "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=1432,
    use_rslora=False,
    loftq_config=None,
)
# 检查模型结构，确认 LoRA 适配器已添加
print(model)

Unsloth 2025.8.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 1536, padding_idx=151654)
        (layers): ModuleList(
          (0): Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1536, out_features=1536, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=1536, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
       

### 6. 配置 SFTTrainer

`SFTTrainer` (Supervised Fine-tuning Trainer) 是一个专门用于指令微调的训练器。我们需要配置 `TrainingArguments` 来指定所有的训练参数，如批量大小、学习率、优化器等。

In [15]:
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 64,
        gradient_accumulation_steps = 2,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 1432,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/20171 [00:00<?, ? examples/s]

### 7. 开始训练

一切准备就绪后，调用 `trainer.train()` 即可开始微调过程。训练结束后，会返回包含训练统计信息（如训练损失）的对象。

In [16]:
trainer_stats = trainer.train()

# 打印训练统计信息
print(trainer_stats)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,171 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 64 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (64 x 2 x 1) = 128
 "-____-"     Trainable parameters = 18,464,768 of 1,795,552,768 (1.03% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.161000
2,3.168900
3,3.078200
4,3.166600
5,3.150200
6,3.008500
7,2.984700
8,2.946300
9,2.900400
10,2.794600


TrainOutput(global_step=60, training_loss=2.4793893059094745, metrics={'train_runtime': 795.3904, 'train_samples_per_second': 9.656, 'train_steps_per_second': 0.075, 'total_flos': 7.163728311484416e+16, 'train_loss': 2.4793893059094745})


### 8. 保存微调后的模型（Lora）

训练完成后，您可以再次进行推理，比较微调后的模型与原始模型的差异。如果对结果满意，可以使用 `model.save_pretrained("your_lora_adapter_path")` 来保存训练好的 LoRA 适配器。

In [17]:
model.save_pretrained("qwen-1.5b_lora_model")

In [18]:
tokenizer.save_pretrained("qwen-1.5b_lora_model")

('qwen-1.5b_lora_model/tokenizer_config.json',
 'qwen-1.5b_lora_model/special_tokens_map.json',
 'qwen-1.5b_lora_model/chat_template.jinja',
 'qwen-1.5b_lora_model/tokenizer.json')

In [20]:
# 模型保存方式二选一（要么使用上面的分开保存，要么使用这里的合并 Lora 保存）
# model.save_pretrained_merged("qwen-1.5b_lora_model", tokenizer, save_method="merged_16bit")

### 9. 测试训练后的生成结果

In [19]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

question="一个患有急性阑尾炎的病人已经发病5天，腹痛稍有减轻但仍然发热，在体检时发现右下腹有压痛的包块，此时应如何处理？", # Question
inputs = tokenizer([inference_prompt.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1000,
    use_cache=True,
)

In [20]:
output = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(output[0].split("### Response:")[1])


<think>
这位病人，5天已经得了急性阑尾炎，还有一点腹痛和发热，但还是有压痛包块。哎，这个情况还挺麻烦的。我得好好想想怎么处理。

首先，压痛包块通常就是阑尾炎的一种表现，所以要先排除其他可能。但这个病人没有明显的感染史，也没有其他症状，只能先看这个包块是不是阑尾炎的表现。

然后，考虑到急性阑尾炎的常见病程，可能得看看其他可能的诊断。比如，急性阑尾炎有时候也会有急性腹痛和发热，所以可能需要考虑急性腹痛和发热。

再想想，如果压痛包块是由于急性阑尾炎引起的，那可能需要先处理这个包块。因为急性阑尾炎导致的包块通常比较严重，所以先处理好这个包块可能是有必要的。

不过，如果病人还有其他症状，比如急性腹痛和发热，可能需要先处理急性腹痛和发热。因为急性腹痛和发热有时候也会引发其他问题，比如急性阑尾炎。

哦，对了，有时候急性阑尾炎可能还会导致急性腹痛和发热，所以得先考虑这些症状。

然后，我要考虑急性阑尾炎的常见病程。急性阑尾炎通常有急性腹痛和发热，这可能与急性阑尾炎有关。

哦，对了，如果病人出现急性腹痛和发热，可能需要先处理急性腹痛和发热，然后再考虑急性阑尾炎。

所以，总结一下，病人现在有急性阑尾炎的表现，包括压痛包块，还有急性腹痛和发热。所以，应该先处理急性腹痛和发热，然后再考虑急性阑尾炎。

这样分步骤来处理，才能确保治疗的合理性和有效性。
</think>
根据病人的情况，病人已经患有急性阑尾炎，并且在5天内出现了腹痛、发热和压痛包块。这种情况下，首先应该处理急性腹痛和发热，以减少对急性阑尾炎的潜在风险。处理急性腹痛和发热通常需要使用药物或手术来解除腹痛和发热。在处理好急性腹痛和发热后，再考虑处理急性阑尾炎的压痛包块。由于急性阑尾炎可能导致压痛包块，因此处理急性腹痛和发热是必要的步骤。建议在急性腹痛和发热被处理后，再进行进一步的急性阑尾炎诊断和治疗。


In [21]:
def generate_response(question: str, model, tokenizer, inference_prompt: str, max_new_tokens: int = 1024) -> str:
    """
    使用指定的模型和分词器为给定的医学问题生成响应。

    Args:
        question (str): 需要模型回答的医学问题。
        model: 已加载的 Unsloth/Hugging Face 模型。
        tokenizer: 对应的分词器。
        inference_prompt (str): 用于格式化输入的 f-string 模板。
        max_new_tokens (int, optional): 生成响应的最大 token 数量。默认为 1024。

    Returns:
        str: 模型生成的响应文本，已去除 prompt 部分。
    """
    # 1. 使用模板格式化输入
    prompt = inference_prompt.format(
        question, # 填充问题
        "",       # 留空，让模型生成 CoT 和 Response
    )

    # 2. 将格式化后的 prompt 进行分词，并转移到 GPU
    inputs = tokenizer([prompt], return_tensors="pt").to(model.device)

    # 3. 使用模型生成输出
    # use_cache=True 用于加速解码过程
    outputs = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=max_new_tokens,
        use_cache=True,
    )
    
    # 4. 将生成的 token 解码为文本
    # skip_special_tokens=True 会移除像 EOS_TOKEN 这样的特殊标记
    decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # 5. 切分字符串，只返回 "### Response:" 之后的部分
    # 使用 .split() 分割并获取响应内容，.strip() 用于去除可能存在的前后空白字符
    response_part = decoded_output.split("### Response:")
    if len(response_part) > 1:
        return response_part[1].strip()
    else:
        # 如果模型没有生成 "### Response:" 标记，则返回整个生成内容以供调试
        return decoded_output

In [22]:
my_question = "对于一名60岁男性患者，出现右侧胸疼并在X线检查中显示右侧肋膈角消失，诊断为肺结核伴右侧胸腔积液，请问哪一项实验室检查对了解胸水的性质更有帮助？"

response = generate_response(my_question, model, tokenizer, inference_prompt)
print("==================== 模型回答 ====================")
print(response)

==================== 模型回答 ====================
<think>
这个60岁男性的症状让我有点担心。他出现的右侧胸疼和X线检查显示右侧肋膈角消失，这让我想到可能的肺结核。

肺结核是通过肺泡结核病引起的一种严重呼吸系统疾病。它会导致肺部肿大，可能引起胸腔积液。

根据这些症状，我应该考虑进行胸腔积液的检查。这会让我更清楚地了解胸腔里积液的性质。

此外，肺结核可能还会引起肺泡结核病，这可能与胸腔积液有关。

因此，胸腔积液的检查对于了解肺结核的表现和影响是很有帮助的。
</think>
对于60岁男性患者出现右侧胸疼并在X线检查中显示右侧肋膈角消失，诊断为肺结核伴右侧胸腔积液的情况，胸腔积液的检查对于了解胸腔积液的性质是有帮助的。

肺结核是一种通过肺泡结核病引起的大气性肺部疾病，会导致肺部肿大、肺泡结核病和胸腔积液。胸腔积液的检查有助于评估患者的肺部肿大程度，了解胸腔积液的性质，以及评估肺部的肺泡结构变化，从而有助于诊断和治疗。

因此，胸腔积液的检查对于了解肺结核伴胸腔积液患者的症状和病变是有帮助的。


In [23]:
my_question = "对于一名 28 岁的男性患者，工作是程序员，常年熬夜，最近突然感觉头晕目眩，甚至有点恶心。请问有可能是什么疾病？"

response = generate_response(my_question, model, tokenizer, inference_prompt)
print("==================== 模型回答 ====================")
print(response)

==================== 模型回答 ====================
<think>
患者是一名28岁的男性，工作是程序员，长期熬夜，最近突然感觉头晕目眩，甚至有点恶心。这种症状让我想到可能是脑力损伤导致的脑力损伤。但根据医生的诊断，这种情况最有可能是脑力损伤，尤其是长时间的脑力劳动，导致的脑力损伤。

首先，脑力损伤在脑力劳动中是很常见的表现，特别是在长时间的脑力劳动中，尤其是长时间的脑力劳动，脑力损伤会逐渐发展到严重的程度。例如，长时间的脑力劳动会导致脑力损伤，从而引发脑力损伤综合征。

脑力损伤综合征是脑力劳动的常见表现，它通常在脑力劳动中发展到一定程度。如果患者的脑力劳动持续时间过长，导致脑力损伤，那么脑力损伤综合征就有可能出现。

因此，脑力损伤综合征的出现是符合患者当前的症状的。患者长期的脑力劳动导致脑力损伤，脑力损伤综合征的出现也符合患者的症状。

因此，根据患者的症状和脑力劳动的持续时间，脑力损伤综合征是这种情况的合理解释。

最终，脑力损伤综合征是患者症状最可能的原因。
</think>
根据患者的症状和脑力劳动的持续时间，脑力损伤综合征是患者当前症状最可能的原因。患者长期的脑力劳动导致脑力损伤，脑力损伤综合征通常在脑力劳动中发展到一定程度。因此，脑力损伤综合征是患者症状最可能的原因。
